# 2.3自动求梯度

In [10]:
import torch
print(torch.__version__)

1.6.0


## TENSOR

In [11]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [12]:
y = x + 2
print(y)

z = y * y * 3
print(z)

out = z.mean()
print(out)


tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


## 梯度

In [13]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [14]:
# grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_() # 梯度数据清零
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


In [15]:
# 关于对不是标量的求导
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

v = torch.tensor([[1.0, 1.0], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)
tensor([2.0000, 2.0000, 0.0200, 0.0020])


In [20]:
# 中断梯度追踪
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
print(x, x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

y3.backward()
print(x.grad)

tensor(1., requires_grad=True) True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True
tensor(2.)


In [21]:
# 对比
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
y2 = x ** 3
y3 = y1 + y2
print(x, x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # True
print(y3, y3.requires_grad) # True

y3.backward()
print(x.grad)

tensor(1., requires_grad=True) True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1., grad_fn=<PowBackward0>) True
tensor(2., grad_fn=<AddBackward0>) True
tensor(5.)


In [22]:
# 只改变其data值，而不记录在autograd中
x = torch.ones(1, requires_grad=True)
print(x.data)
print(x.data.requires_grad) # data的独立于autograd之外

y = 2 * x
x.data *= 100

y.backward()
print(x)
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
